In [73]:
import pandas as pd
import numpy as np
import sympy as sp
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

In [74]:
# Fetch dataset 
#wine_quality = fetch_ucirepo(id=186) 
   
#X = wine_quality.data.features 
#y = wine_quality.data.targets 
  
#print(wine_quality.variables)

In [75]:
red_wine_df = pd.read_csv("winequality-red.csv", delimiter=";")
white_wine_df = pd.read_csv("winequality-white.csv", delimiter=";")

wine_df = pd.concat([red_wine_df, white_wine_df], axis=0)

X = wine_df.drop(columns=['quality'])
y = wine_df['quality']

print(wine_df)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4              0.70         0.00             1.9      0.076   
1               7.8              0.88         0.00             2.6      0.098   
2               7.8              0.76         0.04             2.3      0.092   
3              11.2              0.28         0.56             1.9      0.075   
4               7.4              0.70         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
4893            6.2              0.21         0.29             1.6      0.039   
4894            6.6              0.32         0.36             8.0      0.047   
4895            6.5              0.24         0.19             1.2      0.041   
4896            5.5              0.29         0.30             1.1      0.022   
4897            6.0              0.21         0.38             0.8      0.020   

      free sulfur dioxide  

In [76]:
# Spliting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Converting data to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = y_train.ravel()

In [77]:
# Function for calculating residual sum of squares
def residual_sum_of_squares(y_real, y_pred):
    RSS = np.sum((y_real - y_pred) ** 2)
    return RSS

# Function for calculating gradient of residual sum of squares
def compute_gradient(X, y_real, y_pred):
    m = len(y_real)
    error = y_real - y_pred
    transposed_X = X.transpose()
    gradient = -2 * transposed_X.dot(error) / m

    return gradient

# Initialize parameters
a1 = 0.001  # Learning rate 1
a2 = 0.01 # Learning rate 2
a3 = 0.1 # Learning rate 3
number_of_iterations = 10000  # Number of iterations
num_row = X_train.shape[0]
num_col = X_train.shape[1]

X_train = np.c_[np.ones(num_row), X_train] # Add bias term to X_train (vector of ones is concataneted to X_train from the left)
w = np.random.randn(num_col + 1) * 0.01 # Initialize weights randomly but mutiply with 0.01 to get smaller values
w1 = w2 = w3 = w

# Gradient descent algorithm
for i in range(number_of_iterations):
    
    y_pred_train_1 = X_train.dot(w1)
    y_pred_train_2 = X_train.dot(w2)
    y_pred_train_3 = X_train.dot(w3)

    RSS_1 = residual_sum_of_squares(y_train, y_pred_train_1) # Calculate RSS for w1
    
    RSS_gradient_1 = compute_gradient(X_train, y_train, y_pred_train_1)
    RSS_gradient_2 = compute_gradient(X_train, y_train, y_pred_train_2)
    RSS_gradient_3 = compute_gradient(X_train, y_train, y_pred_train_3)

    
    w1 -= a1 * RSS_gradient_1 # Update weights w1
    w2 -= a2 * RSS_gradient_2 # Update weights w2
    w3 -= a3 * RSS_gradient_3 # Update weights w3


    # Print cost every 1000 iterations
    if i % 1000 == 0:
        print(f"Iteration {i}, Cost: {RSS_1}")

print(f"Final weights with a1 = 0.001: {w1}")
print(f"Final weights with a2 = 0.01: {w2}")
print(f"Final weights with a3 = 0.1: {w3}")


Iteration 0, Cost: 178718.70830576826
Final weights with a1 = 0.001: [ 5.81450837  0.10170125 -0.21905526 -0.02080019  0.22033506 -0.01159569
  0.12250196 -0.14950612 -0.17726483  0.07704958  0.12015313  0.32246666]
Final weights with a2 = 0.01: [ 5.81450837  0.10170125 -0.21905526 -0.02080019  0.22033506 -0.01159569
  0.12250196 -0.14950612 -0.17726483  0.07704958  0.12015313  0.32246666]
Final weights with a3 = 0.1: [ 5.81450837  0.10170125 -0.21905526 -0.02080019  0.22033506 -0.01159569
  0.12250196 -0.14950612 -0.17726483  0.07704958  0.12015313  0.32246666]


In [78]:
# Scaling X_test with scaler fitted to X_train
X_test = scaler.transform(X_test)

# Converting to numpy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)
y_test = y_test.ravel()

# Adding bias term to X_test
num_row = X_test.shape[0]
X_test = np.c_[np.ones(num_row), X_test]

# Making predictions for X_test
y_pred_test_1 = X_test.dot(w1)
y_pred_test_2 = X_test.dot(w2)
y_pred_test_3 = X_test.dot(w3)

In [79]:
# Measures of performance
mae = [ mean_absolute_error(y_test, y_pred_test_1), mean_absolute_error(y_test, y_pred_test_2), mean_absolute_error(y_test, y_pred_test_3)] 
mse = [ mean_squared_error(y_test, y_pred_test_1), mean_squared_error(y_test, y_pred_test_2), mean_squared_error(y_test, y_pred_test_3)]
rmse = np.sqrt(mse)
r2 = [r2_score(y_test, y_pred_test_1), r2_score(y_test, y_pred_test_2), r2_score(y_test, y_pred_test_3)]

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error (MAE): [0.5658710072506614, 0.5658710072506614, 0.5658710072506614]
Mean Squared Error (MSE): [0.5466958511495601, 0.5466958511495601, 0.5466958511495601]
Root Mean Squared Error (RMSE): [0.73938884 0.73938884 0.73938884]
R-squared: [0.2597681129398879, 0.2597681129398879, 0.2597681129398879]


In [80]:
# Using the implemented gradient descent algorithm from sklearn library
sgd_regressor = SGDRegressor(max_iter=10000, tol=1e-3, learning_rate='constant', eta0=0.001, random_state=42)
sgd_regressor.fit(X_train, y_train)
y_pred_test_sklearn = sgd_regressor.predict(X_test)

# Metrics of performance
mae = mean_absolute_error(y_test, y_pred_test_sklearn)
mse = mean_squared_error(y_test, y_pred_test_sklearn)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_test_sklearn)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error (MAE): 0.568514098413171
Mean Squared Error (MSE): 0.5541035332628875
Root Mean Squared Error (RMSE): 0.74438130904993
R-squared: 0.2497380340615507


In [81]:
# Random Forest regression

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
y_pred_test_rf = rf_regressor.predict(X_test)

# Metrics of performance
mae_rf = mean_absolute_error(y_test, y_pred_test_rf)
mse_rf = mean_squared_error(y_test, y_pred_test_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_test_rf)

print(f"Mean Absolute Error (MAE): {mae_rf}")
print(f"Mean Squared Error (MSE): {mse_rf}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf}")
print(f"R-squared: {r2_rf}")

Mean Absolute Error (MAE): 0.43814615384615385
Mean Squared Error (MSE): 0.3696033076923077
Root Mean Squared Error (RMSE): 0.6079500865139404
R-squared: 0.4995532646874079
